# Training using numpy arrays
In this notebook we'll take a look at how to use numpy data to train a neural network in CNTK.
The model used here is a basic classification model that uses random training samples. The goal is to demonstrate how to use numpy, so we left out the stuff that you don't need.

## Generating sample data
Before we can actually train the model, we need to generate some sample data. We used the numpy random number generator for that. Notice that, we used a random integer for the classes with `high=2` because we want numbers between 0 and 1 (2 classes).
We converted both to `float32` so that we can use the data without warnings in CNTK.

In [1]:
import numpy as np

num_samples = 20000

y = np.random.randint(size=(num_samples, 1), low=0, high=2).astype(np.float32)
X = np.random.random(size=(num_samples, 4)).astype(np.float32)

## Building the model
The model is a basic classification model. We use one hidden layer and an output layer. 
Both have a sigmoid activation function. 

In [1]:
from cntk.layers import Dense, Sequential
from cntk import input_variable, default_options
from cntk.ops import sigmoid

with default_options(activation=sigmoid):
    model = Sequential([
        Dense(4),
        Dense(1)
    ])

features = input_variable(4)
labels = input_variable(1)

z = model(features)

## Training the neural network
To train the neural network you can invoke the `train` method on the loss function to feed the numpy data in chunks. These chunks are called minibatches. 

We've attached a progress printer to the training process so you can see what's going on.
Also note, that we used a `minibatch_size` setting to control the number of samples per minibatch.

In [3]:
from cntk.learners import sgd
from cntk.losses import binary_cross_entropy
from cntk.logging import ProgressPrinter

progress_writer = ProgressPrinter(0)
learner = sgd(z.parameters, lr=0.1)
loss = binary_cross_entropy(z, labels)

training_summary = loss.train((X,y), parameter_learners=[learner], callbacks=[progress_writer], minibatch_size=512)

 average      since    average      since      examples
    loss       last     metric       last              
 ------------------------------------------------------
Learning rate per minibatch: 0.1
    0.996      0.996          0          0           512
    0.946       0.92          0          0          1536
    0.902      0.869          0          0          3584
    0.843      0.792          0          0          7680
    0.782      0.725          0          0         15872
